In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 40.3 MB/s 
     |████████████████████████████████| 6.5 MB 38.6 MB/s 
     |████████████████████████████████| 895 kB 29.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 582 kB 5.1 MB/s 
     |████████████████████████████████| 398 kB 59.9 MB/s 
     |████████████████████████████████| 136 kB 64.8 MB/s 
     |████████████████████████████████| 1.1 MB 47.1 MB/s 
     |████████████████████████████████| 144 kB 50.5 MB/s 
     |████████████████████████████████| 271 kB 45.5 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


In [ ]:
import json
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer)
from tqdm.auto import tqdm

In [ ]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format="retina"
sns.set(style="whitegrid",palette="muted", font_scale=1.2)
rcParams["figure.figsize"]=16,10

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer)
from tqdm.auto import tqdm

In [ ]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format="retina"
sns.set(style="whitegrid",palette="muted", font_scale=1.2)
rcParams["figure.figsize"]=16,10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
  
# Opening JSON file
with open('./drive/MyDrive/Data/personachat.json') as json_file:
    personachat = json.load(json_file)
  

In [ ]:
conversations=[]
for dialog in personachat["train"]:
  dialogue_persona=dialog["personality"]
  utterance=dialog["utterances"][-1]
  conv=utterance["history"]
  if conv[0]=="__ SILENCE __":
    conv=conv[1:]
  personality=" ".join(x for x in dialogue_persona)
  conv=[personality]+conv
  conversations.append(conv)


In [ ]:
conversations_test=[]
for dialog in personachat["valid"]:
  dialogue_persona=dialog["personality"]
  utterance=dialog["utterances"][-1]
  conv=utterance["history"]
  if conv[0]=="__ SILENCE __":
    conv=conv[1:]
  personality=" ".join(x for x in dialogue_persona)
  conv=[personality]+conv
  conversations_test.append(conv)


In [ ]:
conversations_test[-1]

['i am a vegan and i love hummus . i love rollercoasters and sky diving . i do like watching cooking shows . i am not a good swimmer at all .',
 'hi how are you ?',
 'i am doing great on this crisp fall night',
 'do you like sports ? i love tennis and play it for a living .',
 'sports are ok but i love cooking shows and cooking',
 'my husband is a doctor so i cook meals and freeze them to have on hand .',
 'that is a great idea i am single so i only make enough for one',
 'well enjoy being single , we are growing our family so free time is short .',
 'but it is tough cause i am vegan and my friends are not',
 'bummer , i take my dog for long walks to de stress , how about you ?',
 'i like to go skydiving or ride a rollercoaster',
 'when its raining or too cold we walk the stairs in our home , three stories !',
 'wow first floor apt for me so no stairs',
 'with a big family we need the room , plus the dog .',
 'how many children do u have',
 'three and one on the way . the dog is like a

In [ ]:
def construct_conv(row, tokenizer, eos = True):
    # from: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
    flatten = lambda l: [item for sublist in l for item in sublist]
    #for x in row:
      #print(list(reversed(tokenizer.encode(x)+ [tokenizer.eos_token_id])) )
    conv = list([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row])
    # return a list of new sentences : zhere for each previous sentence row we will add a token that indicate that the current sentence is ended 
    conv = flatten(conv) # make all the rows as a seauence conversation by flatten all the row cells int a one cell that contains the sequence of sentences
    return conv

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

#-----------------

In [ ]:
number_of_dialogues=len(personachat["train"])
number_of_dialogues

17878

In [ ]:
conversations=[]
labels=[]
for dialog in personachat["train"]:
  dialogue_persona=dialog["personality"]
  utterance=dialog["utterances"][-1]
  conv=utterance["history"]
  if conv[0]=="__ SILENCE __":
    conv=conv[1:]
  personality=" ".join(x for x in dialogue_persona)
  labels.append(personality)
  for c in conv:
    text_conv=" ".join(c for c in conv)
  conversations.append(text_conv)
conversations_dict={"text":conversations,"summary":labels}


In [ ]:
conversations_test=[]
labels_test=[]
for dialog in personachat["valid"]:
  dialogue_persona=dialog["personality"]
  utterance=dialog["utterances"][-1]
  conv=utterance["history"]
  if conv[0]=="__ SILENCE __":
    conv=conv[1:]
  personality=" ".join(x for x in dialogue_persona)
  labels_test.append(personality)
  for c in conv:
    text_conv=" ".join(c for c in conv)
  conversations_test.append(text_conv)
conversations_test_dict={"text":conversations_test,"summary":labels_test}

In [ ]:
conversations_test_dict["text"][0]

"hello what are doing today ? i am good , i just got off work and tired , i have two jobs . i just got done watching a horror movie i rather read , i've read about 20 books this year . wow ! i do love a good horror movie . loving this cooler weather but a good movie is always good . yes ! my son is in junior high and i just started letting him watch them too i work in the movies as well . neat ! ! i used to work in the human services field yes it is neat , i stunt double , it is so much fun and hard work . yes i bet you can get hurt . my wife works and i stay at home nice , i only have one parent so now i help out my mom . i bet she appreciates that very much . she raised me right , i am just like her . my dad was always busy working at home depot"

In [ ]:
df_train=pd.DataFrame(conversations_dict)

In [ ]:
test_df=pd.DataFrame(conversations_test_dict)

In [ ]:
"""train_conv=conversations[:12000]
eval_conv=conversations[12000:]
test_conv=conversations_test"""

'train_conv=conversations[:12000]\neval_conv=conversations[12000:]\ntest_conv=conversations_test'

In [ ]:
train_df, eval_df= train_test_split(df_train,test_size=0.1)


In [ ]:
train_df.iloc[0]["summary"]

'i have an older brother . i live in the city . i work in a bookstore . i love my girlfriend . i dyed my hair pink .'

In [ ]:
test_df.iloc[0]['text']

"hello what are doing today ? i am good , i just got off work and tired , i have two jobs . i just got done watching a horror movie i rather read , i've read about 20 books this year . wow ! i do love a good horror movie . loving this cooler weather but a good movie is always good . yes ! my son is in junior high and i just started letting him watch them too i work in the movies as well . neat ! ! i used to work in the human services field yes it is neat , i stunt double , it is so much fun and hard work . yes i bet you can get hurt . my wife works and i stay at home nice , i only have one parent so now i help out my mom . i bet she appreciates that very much . she raised me right , i am just like her . my dad was always busy working at home depot"

In [ ]:
train_df.shape, test_df.shape

((16090, 2), (1000, 2))

In [ ]:
test_df

,text,summary
0,"hello what are doing today ? i am good , i jus...",i read twenty books a year . i'm a stunt doubl...
1,hi ! how are you doing tonight ? i'm doing gre...,i have two dogs . i like to work on vintage ca...
2,"hello , how are you doing ? i love spending ti...",i love disneyland and mickey mouse . i love to...
3,hi how are you doing ? i am okay how about you...,i love to drink fancy tea . i have a big libra...
4,hi i dye my hair 4 times a year do you live in...,i'm a little girl . i've superpowers . i like ...
...,...,...
995,hello i am a country guy who wants to move to ...,my husband is adopted . i've a german shepherd...
996,hello ! how are you this evening ? pretty good...,i produce music for artists . i drive a 2015 h...
997,my parents are teachers and together we love b...,i'm terrified of scorpions . i'm employed by t...
998,"how are you doing today doing good man , just ...",i am a musician and hope to make it big some d...


In [ ]:
MODEL_NAME="t5-base"
tokenizer=T5Tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

In [ ]:
def tokenize_inputs(df,tokenizer, num_embeddings=1024):
    
    outputs = tokenizer.batch_encode_plus(list(df['text'].values),truncation=True,return_tensors='pt',return_attention_mask=True,max_length=num_embeddings, padding='max_length')
    labels_to = tokenizer.batch_encode_plus(list(df["summary"].values),truncation=True,return_tensors='pt',return_attention_mask=True,max_length=100, padding='max_length')
    # convert tokenized text into numeric ids for the appropriate LM
    input_ids =outputs.input_ids
    

    return (input_ids,outputs.attention_mask,labels_to.input_ids,labels_to.attention_mask)
    

In [ ]:
train_input_ids,train_attention_masks,train_labels,train_labels_attention=tokenize_inputs(train_df, tokenizer=tokenizer, num_embeddings=512) #1024


In [ ]:
test_input_ids,test_attention_masks,test_labels,test_labels_attention=tokenize_inputs(test_df, tokenizer=tokenizer, num_embeddings=512)

In [ ]:
valid_input_ids,valid_attention_masks,valid_labels,valid_labels_attention=tokenize_inputs(eval_df, tokenizer=tokenizer, num_embeddings=512)

In [ ]:
#X_train,train_masks,Y_train = (train_input_ids,train_attention_masks,torch.tensor(y_train_vector,dtype=torch.float32))
X_train,train_masks,Y_train,Y_train_attention= (train_input_ids,train_attention_masks,train_labels,train_labels_attention)

X_valid,valid_masks,Y_valid,Y_valid_attention= (valid_input_ids,valid_attention_masks,valid_labels,valid_labels_attention)

In [ ]:
import os
import math
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

In [ ]:
batchsize=5

train_data = TensorDataset(X_train, train_masks, Y_train,Y_train_attention)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,\
                              sampler=train_sampler,\
                              batch_size=batchsize)

validation_data = TensorDataset(X_valid, valid_masks, Y_valid, Y_valid_attention)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,\
                                   sampler=validation_sampler,\
                                   batch_size=batchsize)

In [ ]:
X_test,test_masks,Y_test,Y_test_attention= (test_input_ids,test_attention_masks,test_labels,test_labels_attention)


In [ ]:
batchsize=5
testing_data = TensorDataset(X_test, test_masks, Y_test,Y_test_attention)
test_sampler = SequentialSampler(testing_data)
test_dataloader = DataLoader(testing_data,\
                                   sampler=test_sampler,\
                                   batch_size=batchsize)

In [ ]:
class T5ProfileGeneration(torch.nn.Module):
  
  def __init__(self,tokenizer):
    super(T5ProfileGeneration, self).__init__()
    
    self.t5 = self.model=T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

   

   
  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels=None,decoder_attention_mask=None): 
   
        
    output=self.t5(input_ids,attention_mask=attention_mask,labels=labels,decoder_attention_mask=decoder_attention_mask)
      
      
    return output.loss,output.logits


  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state
    
  

# len(Y_train[0]) = 6

In [ ]:
import torch
device =torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

In [ ]:
model = T5ProfileGeneration(tokenizer=tokenizer)
model.to(device)

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

T5ProfileGeneration(
  (t5): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_f

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01, correct_bias=False)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:

#Define the training function : this code is copied from https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

def train(model, num_epochs,\
          optimizer,\
          train_dataloader, valid_dataloader,\
          model_save_path,\
          train_loss_set=[], valid_loss_set = [],\
          lowest_eval_loss=None, start_epoch=0,\
          device="cpu"
          ):
  """
  Train the model and save the model with the lowest validation loss
  """

  model.to(device)

  for i in trange(num_epochs, desc="Epoch"):
    
    actual_epoch = start_epoch + i

    

    model.train()

    tr_loss = 0
    num_train_samples = 0

    for step, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_topics,decoder_attention_mask = batch
      # Clear out the gradients (by default they accumulate)
      optimizer.zero_grad()
      # Forward pass
      loss,_ = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels_topics,decoder_attention_mask=decoder_attention_mask)
      # store train loss
      tr_loss += loss.item()
      num_train_samples += b_labels_topics.size(0)
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      #scheduler.step()

    # Update tracking variables
    epoch_train_loss = tr_loss/num_train_samples
    train_loss_set.append(epoch_train_loss)

    print("Train loss: {}".format(epoch_train_loss))

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss = 0
    num_eval_samples = 0

    # Evaluate data for one epoch
    for batch in valid_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_topics,decoder_attention_mask = batch
      # Telling the model not to compute or store gradients,
      # saving memory and speeding up validation
      with torch.no_grad():
        # Forward pass, calculate validation loss
        loss,_= model(b_input_ids, attention_mask=b_input_mask, labels=b_labels_topics,decoder_attention_mask=decoder_attention_mask)
        # store valid loss
        eval_loss += loss.item()
        num_eval_samples += b_labels_topics.size(0)

    epoch_eval_loss = eval_loss/num_eval_samples
    valid_loss_set.append(epoch_eval_loss)

    print("Valid loss: {}".format(epoch_eval_loss))
    """last_path='drive/MyDrive/Models/DialogGeneration/pretrained/TGD_DiallogGPT_PersonalCode_Big5_100epochs_1DataAugmentation.bin'
    
    #model.dialogGPT.save_pretrained('drive/MyDrive/Models/DialogGeneration/pretrained/TGD_DiallogGPT_PersonalCode_Big5_100epochs_1DataAugmentation')#TGD_Classification_Concatenate_Both_Topic_Entent_Sigmoid
    save_model(model, last_path, actual_epoch,\
                 lowest_eval_loss, train_loss_set, valid_loss_set)"""

    if lowest_eval_loss == None:
      lowest_eval_loss = epoch_eval_loss
      # save model
      save_model(model, model_save_path, actual_epoch,\
                 lowest_eval_loss, train_loss_set, valid_loss_set)
    else:
      if epoch_eval_loss < lowest_eval_loss:
        lowest_eval_loss = epoch_eval_loss
        # save model
        save_model(model, model_save_path, actual_epoch,\
                   lowest_eval_loss, train_loss_set, valid_loss_set)
    print("\n")

  return model, train_loss_set, valid_loss_set


def save_model(model, save_path, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist):
  """
  Save the model to the path directory provided
  """
  model_to_save = model.module if hasattr(model, 'module') else model
  checkpoint = {'epochs': epochs, \
                'lowest_eval_loss': lowest_eval_loss,\
                'state_dict': model_to_save.state_dict(),\
                'train_loss_hist': train_loss_hist,\
                'valid_loss_hist': valid_loss_hist
               }
  torch.save(checkpoint, save_path)
  print("Saving model at epoch {} with validation loss of {}".format(epochs,\
                                                                     lowest_eval_loss))
  return
  
def load_model(save_path,tokenizer):
  """
  Load the model from the path directory provided
  """
  checkpoint = torch.load(save_path)
  model_state_dict = checkpoint['state_dict']
  model = DialogGPT(tokenizer)
  model.load_state_dict(model_state_dict)

  epochs = checkpoint["epochs"]
  lowest_eval_loss = checkpoint["lowest_eval_loss"]
  train_loss_hist = checkpoint["train_loss_hist"]
  valid_loss_hist = checkpoint["valid_loss_hist"]
  
  return model, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist

In [ ]:
import os
num_epochs=50

cwd = os.getcwd()

model_save_path = output_model_file = os.path.join(cwd, "drive/MyDrive/Models/Summarization/PresonaChat-ProfileGeneration_whith_t5.bin")


In [ ]:
model, train_loss_set, valid_loss_set = train(model=model,\
                                              num_epochs=num_epochs,\
                                              optimizer=optimizer,\
                                              train_dataloader=train_dataloader,\
                                              valid_dataloader=validation_dataloader,\
                                              model_save_path=model_save_path,\
                                              device=device)
#0.016783231785479146

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Train loss: 0.1617177924786094
Valid loss: 0.14072453660652942


Epoch:   2%|▏         | 1/50 [26:35<21:42:40, 1595.11s/it]

Saving model at epoch 0 with validation loss of 0.14072453660652942


Train loss: 0.1434215950291671
Valid loss: 0.13019938383593122


Epoch:   4%|▍         | 2/50 [53:12<21:17:07, 1596.40s/it]

...

Saving model at epoch 39 with validation loss of 0.052579783528243135


Train loss: 0.048183815868431325
Valid loss: 0.05209872925808232


Epoch:  82%|████████▏ | 41/50 [18:10:55<3:59:28, 1596.48s/it]

Saving model at epoch 40 with validation loss of 0.05209872925808232


Train loss: 0.047355764969544356
Valid loss: 0.05146490201620714


Epoch:  84%|████████▍ | 42/50 [18:37:31<3:32:50, 1596.37s/it]

Saving model at epoch 41 with validation loss of 0.05146490201620714




In [ ]:
#Define the training function : this code is copied from https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

checkpoint = torch.load(model_save_path)
model_state_dict = checkpoint['state_dict']
model = T5ProfileGeneration(tokenizer=tokenizer)
model.load_state_dict(model_state_dict)
model.to(device)

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

T5ProfileGeneration(
  (t5): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_f

In [ ]:
def summarize(text,trained_model):
  text_encoding=tokenizer(text,padding="max_length",max_length=512,truncation=True,return_attention_mask=True,return_tensors="pt")
  generated_ids=trained_model.generate(
      input_ids=text_encoding["input_ids"].to('cuda'),
      attention_mask=text_encoding["attention_mask"].to('cuda'),
      max_length=150, 
      num_beams=15,
      early_stopping=True,
      repetition_penalty=2.5,
      length_penalty=1.0,
  )
  preds=[tokenizer.decode(gen_id,skip_special_tokens=True,clean_up_tokenization_space=True) for gen_id in generated_ids]
  return " ".join(preds)

In [ ]:
!pip install datasets
import datasets

     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 70.2 MB/s 
     |████████████████████████████████| 127 kB 51.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install rouge_score

In [ ]:
test_df

,text,summary
0,"hello what are doing today ? i am good , i jus...",i read twenty books a year . i'm a stunt doubl...
1,hi ! how are you doing tonight ? i'm doing gre...,i have two dogs . i like to work on vintage ca...
2,"hello , how are you doing ? i love spending ti...",i love disneyland and mickey mouse . i love to...
3,hi how are you doing ? i am okay how about you...,i love to drink fancy tea . i have a big libra...
4,hi i dye my hair 4 times a year do you live in...,i'm a little girl . i've superpowers . i like ...
...,...,...
995,hello i am a country guy who wants to move to ...,my husband is adopted . i've a german shepherd...
996,hello ! how are you this evening ? pretty good...,i produce music for artists . i drive a 2015 h...
997,my parents are teachers and together we love b...,i'm terrified of scorpions . i'm employed by t...
998,"how are you doing today doing good man , just ...",i am a musician and hope to make it big some d...


In [ ]:
def get_responses(pretrained,rows):
  all_rows=[]
  model=pretrained
  all_responses=[]
  
  for k,dialogue in enumerate(rows):
    print(k)
    current_data=""
    summary=summarize(dialogue,model)
    
    all_responses.append(summary)

  return all_responses
all_summaries=get_responses(model.t5,test_df['text'].values)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
rouge = datasets.load_metric("rouge")


In [ ]:
all_real_summaries=list(test_df['summary'].values)
rouge_output = rouge.compute(predictions=all_summaries, references=all_real_summaries, rouge_types=["rouge2"])["rouge2"].mid
print(f" \n rouge2_precision {round(rouge_output.precision, 4)}\n rouge2_recall {round(rouge_output.recall, 4)} \n rouge2_f {round(rouge_output.fmeasure, 4)} \n"
)

 
 rouge2_precision 0.1155
 rouge2_recall 0.0983 
 rouge2_f 0.1037 



In [ ]:
'''all_real_summaries=list(test_df['text'].values)
rouge_output = rouge.compute(predictions=all_summaries, references=all_real_summaries, rouge_types=["rouge2"])["rouge2"].mid
print(f" \n rouge2_precision {round(rouge_output.precision, 4)}\n rouge2_recall {round(rouge_output.recall, 4)} \n rouge2_f {round(rouge_output.fmeasure, 4)} \n"
)'''

 
 rouge2_precision 0.2263
 rouge2_recall 0.0368 
 rouge2_f 0.0627 



In [ ]:
predicted_responses_splited=[x.split() for x in all_summaries]
predicted_responses_splited

[["i've",
  'one',
  'son.',
  'i',
  'like',
  'to',
  'read',
  'horror',
  'novels.',
  'my',
  'wife',
  'stays',
  'home',
  'with',
  'our',
  'kids.',
  'i',
  'work',
  'two',
  'jobs',
  'as',
  'a',
  'cashier.'],
 ['i',
  'like',
  'to',
  'work',
  'on',
  'cars.',
  'i',
  'like',
  'to',
  'listen',
  'to',
  'country',
  'music.',
  'i',
  'like',
  'to',
  'work',
  'on',
  'cars.',
  'i',
  'like',
  'to',
  'work',
  'on',
  'cars.'],
 ['i',
  'enjoy',
  'spending',
  'time',
  'with',
  'my',
  'family.',
  'i',
  'am',
  'married',
  'and',
  'have',
  '2',
  'children.',
  'i',
  'walk',
  'three',
  'miles',
  'home',
  'every',
  'night.',
  'i',
  'work',
  'as',
  'a',
  'babysitter.'],
 ['i',
  'drink',
  '10',
  'cups',
  'of',
  'tea',
  'every',
  'day.',
  'i',
  'can',
  't',
  'get',
  'enough',
  'ginger',
  'snaps.',
  'i',
  'like',
  'to',
  'keep',
  'my',
  'hair',
  'cut',
  'very',
  'short.',
  'i',
  'can',
  't',
  'get',
  'enough',
  'ginger

In [ ]:
real_responses_splited=[[x.split() ]for x in all_real_summaries]


In [ ]:
from datasets import load_metric
blue = load_metric('bleu')

In [ ]:
metric = load_metric('bleu')

In [ ]:
final_score = metric.compute(predictions=predicted_responses_splited, references=real_responses_splited)

In [ ]:
final_score
#10 epochs without role : 0.06510915358100344
#10 epochs with role : 0.28978481910617165

{'bleu': 0.03564741565011465,
 'brevity_penalty': 0.7569474684667681,
 'length_ratio': 0.782190203430866,
 'precisions': [0.30473350445417485,
  0.07237564726188608,
  0.025681855299689693,
  0.008683807253533118],
 'reference_length': 33869,
 'translation_length': 26492}

In [ ]:
# prefect match
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu


In [ ]:
#50 Concatenadet_endoftext
print('Individual 1-gram: %f' % corpus_bleu(real_responses_splited, predicted_responses_splited, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % corpus_bleu(real_responses_splited, predicted_responses_splited, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % corpus_bleu(real_responses_splited, predicted_responses_splited, weights=(0, 0, 1, 0)))
#print('Individual 4-gram: %f' % corpus_bleu(real_responses, predicted_responses, weights=(0, 0, 0, 1)))

Individual 1-gram: 0.230667
Individual 2-gram: 0.054785
Individual 3-gram: 0.019440


In [ ]:
!pip install bert_score

     |████████████████████████████████| 60 kB 3.0 MB/s 


In [ ]:
metric = load_metric("bertscore")

In [ ]:
final_score = metric.compute(predictions=all_summaries,references=all_real_summaries,lang="en")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [ ]:
from statistics import mean, median, stdev


In [ ]:
mean(final_score["recall"])
#

0.8601434326767922

In [ ]:
mean(final_score["precision"])
#

0.8844973955750466

In [ ]:
mean(final_score["f1"])
#

0.8720550994873046